# ASTR 21100 

# *"Computational Techniques in Astrophysics"*

## Lab session 1 (Thursday, Jan 16, 2020)

### Instructor: Andrey Kravtsov

### office: ERC 415; email: kravtsov@uchicago.edu
### office hours: Tue, 10:30am-noon (unless noted otherwise)

### Teaching Assistants: 
### Dimitrios Tanoglidis (dtanoglidis@uchicaago.edu)
### Georgios Zakharegkas (gzakharegkas@uchicago.edu)

### Background: from trapezoidal to Simpson integration scheme

One of the simplest methods of numerical integration is *"trapezoidal integration."* It can be used to get reasonably accurate numerical integral estimate and easiest to remember. 

Specifically, to evaluate integral of $f(x)$ (area under $f(x)$ for $x\in[a,b]$):

Split the integration interval into $N$ equal-size sub-intervals of size $h=(b-a)/N=x_{i+1}-x_i$. For convenience, let's denote $f_i = f(x_i)$, $f_{i+1}=f(x_{i+1})$, etc. There will be $N+1$ points at which the integrated function needs to be evaluated, including interval ends $a$ and $b$. 

Approximate area under $f(x)$ in each interval, $A_i$, can then be approximated by the area of the trapezoid formed by vertices $(x_i, 0)$, $(x_{i+1},0)$, $(x_i, f_i)$, $(x_{i+1}, f_{i+1})$, $T_i$:

$$T_i = \frac{1}{2}(f_i + f_{i+1})\, h.$$

The total area under $f(x)$ in the interval $[a,b]$ is then:

$$A = \int\limits_a^b f(x)\, dx = \sum\limits_{i=0}^{N-1} A_i \approx \sum\limits_{i=0}^{N-1} T_i= \frac{h}{2}\sum\limits_{i=0}^{N-1}(f_i + f_{i+1}),$$

which can be recast as

$$A \approx h\left[\frac{1}{2}(f_0 + f_{N}) + \sum\limits_{i=1}^{N-1}f_i,\right]$$

In class on Wednesday, January 15, I showed how trapezoidal scheme can be used to get a scheme of "higher order" - that is a scheme in which fractional error scales as higher power of step size, $h$. 

Namely, if I denote trapezoidal scheme for a step size $h$ as

$$T(h) = \frac{h}{2}\sum\limits_{i=0}^{N-1}(f_i + f_{i+1}) = h\left[\frac{1}{2}(f_0 + f_{N}) + \sum\limits_{i=1}^{N-1}f_i,\right],$$

I showed in class (see <a href="https://github.com/a-kravtsov/a211w20/blob/master/02_integration_approximation.ipynb">02_integration_approximation.ipynb</a> notebook in the github repository) that the following construct

$$R_1(h) = \frac{1}{3}\left[4T(h/2) - T(h)\right]$$

has fractional error that scales as $\epsilon_{R_1}\propto h^4$ instead of $\epsilon_T\propto h^2$ of the trapezoidal scheme. 

Usually, it is said that the scheme $R_1(h)$ has a higher order of accuracy  than the trapezoidal scheme. This means that it will give a more accurate result for a given step size $h$. $R_1(h)$ is actually the integration scheme known as the *Simpson rule*. 

### A more efficient calculation of $R_1(h)$

Computation of $R_1(h)$ involves computation of $T(h)$ and $T(h/2)$ for the same interval and function. 

However, computation of $T(h/2)$ replicates computation of function values at all of the nodes that were used to compute $T(h)$. In other words, computation of $T(h)$ is *embedded* in computation of $T(h/2)$. It is thus inefficient/redundant to compute $T(h)$ and $T(h/2)$ separately via two calls, as I have done in the <a href="https://github.com/a-kravtsov/a211w20/blob/master/02_integration_approximation.ipynb">02_integration_approximation.ipynb</a> notebook. 

If we denote $h_j = (b-a)/2^j$ and trapezoidal scheme with step $h_j$ as $T_j$, then it is easy to show that trapezoidal scheme for $j=1, 2, \ldots$ can be written as

$$T_j = \frac{1}{2} T_{j-1} + h_j\sum\limits_{i=1}^{2^{j-1}}f(a+(2i-1)h_j).$$

This expression allows to reuse the previous computations used to estimate $T_{j-1}$ in computing $T_j$. Taking this into account, the computation of $R_1(h)$ described above, can be written as 

$$R_1(h) = \frac{1}{3}\left[4T(h/2)-T(h)\right]=\frac{1}{3}\left[T(h)+2h\sum\limits_{i=1}^{2^{j-1}}f\left(a+(2i-1)\frac{h}{2}\right)\right],$$

where $j=\log_2[2(b-a)/h]$.

### Tasks for the lab

* Write routine for trapezoidal integration that takes input of the function to be integrated, end points $a$ and $b$ ($b>a$), defining the integration range, and step size $h\leq b-a$, and returns the estimate of the integral obtained using $R_1(h)$ using the efficient formula above. 

First, implement the routine in python the way with which you are most familiar. When you get your routine working and after you test it, try to write another version that uses numpy vector operations and avoiding any explicit loops in computing $R_1(h)$. You can use the routine <tt>trapzd</tt> in the <a href="https://github.com/a-kravtsov/a211w20/blob/master/02_integration_approximation.ipynb">02_integration_approximation.ipynb</a> as an example.  It may be helpful to use numpy functions <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html">numpy.arange</a> and <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html">numpy.sum</a>. The former is useful to set up a vector of $x_i$ values at which to evaluate $f$ and the latter can be used for vectorized summation of the result.  


* Test your routine by comparing its result to the result of direct calculation of the integral for a function, for which the "closed form" expression for the integral exists - that is there is an analytic expression for the integral, such as $\int_0^1 e^x dx$, $\int_0^\pi\sin x dx$, etc.

* *The fractional error* of your integral estimate is computed as
$$\epsilon = \vert 1-R_1(h)/A_{\rm direct}\vert,$$
where $R_1(h)$ is the estimate of the integral using trapezoidal method with step $h$ and $A_{\rm direct}$ is the result of direct calculation of the integral. 

    Compute the fractional error of the test integral for a grid of values of $h$ and record it in an array or list. Make sure your $h$-grid spans sufficiently wide range of values, so that fractional error reaches values at least as low as $\approx 10^{-16}$. 


* Plot the $\log_{10}$ of the fractional error $\epsilon$ as a function of $\log_{10}$ of step size $h$ using routine <tt>plot_line</tt> below. Examine the plot and try answer the following questions:

    1) does the fractional error decrease monotonically with decreasing $h$ for all values of $h$? If not, what do you think prevents it from decreasing? 
    
    2) In the range of $h$ values where fractional error is monotonically decreasing with decreasing $h$, can you figure out the functional form of the dependence of $\epsilon$ on $h$ from your plot?
    
    3) Suppose instead of the test integral for which the correct answer can be calculated using "closed form" expression of the integral, we are dealing with an integral for which not such closed form expression exists. Can you think of a way to evaluate the accuracy of your estimate of the integral in this case? 
    

### Matplotlib

Matplotlib is a package for (mostly) 2D plots built upon the Numpy and Scipy libraries. It was conceived by <a href="https://en.wikipedia.org/wiki/John_D._Hunter">John Hunter</a> at U.Chicago in 2002, developed by him and others over the subsequent decade into a full-fledged library. 


In [1]:
# import matplotlib's PyPlot library denoting it as plt for brevity
import matplotlib.pyplot as plt

In [6]:
def plot_pretty(dpi=175,fontsize=9):
    # import pyplot and set some parameters to make plots prettier
    plt.rc("savefig", dpi=dpi)
    plt.rc("figure", dpi=dpi)
    plt.rc('font', size=fontsize)
    plt.rc('xtick', direction='in') 
    plt.rc('ytick', direction='in')
    plt.rc('xtick.major', pad=5) 
    plt.rc('xtick.minor', pad=5)
    plt.rc('ytick.major', pad=5) 
    plt.rc('ytick.minor', pad=5)
    plt.rc('lines', dotted_pattern = [0.5, 1.1])

    return

plot_pretty(dpi=250, fontsize=9)

### LaTeX

Although not strictly required, I highly recommend installing <a href="https://www.latex-project.org/get/">a LaTeX distribution</a> on your laptop. I use LaTeX commands to format plot labels in matplotlib. 

In [4]:
#if you don't have LaTeX installed on your laptop and this statement 
# generates error, comment it out
plt.rc('text', usetex=True)


In [7]:
plot_pretty(dpi=150, fontsize=12)